In [126]:
# draft_2
# import:
import random
from itertools import product
random.seed(1)

# indexing:
STARTING = 0
DESTINATION = 1
ROW = 0
COLUMN = 1

# variables:
list_position = [(0,0), (0,4)]
num_of_pedestrian = 2
n_rows = 1
n_cols = 5

# draft_2_pedestrian
class Pedestrian():
 
    # def __init__(self, list_position, num_of_pedestrian):
    #     self.list_position = list_position
    #     self.list_sd = self.create_starting_destination(list_position)
    #     self.list_pedestrian_sd = self.create_pedestrian_sd(num_of_pedestrian)
    #     # create_speed

    # staticmethod
    # def create_starting_destination(list_position):
    #     starting_point = list_position # since list mentioned in the init method, can we put self.list_position?
    #     destination_point = list_position
    #     list_sd = []
    #     for starting, destination in product(starting_point, destination_point):
    #         if starting != destination:
    #             list_sd.append((starting, destination))
    #     return list_sd # return a list of tuple

    # @staticmethod
    # def create_pedestrian_sd(num_of_pedestrian):
    #     pedestrian_s = []
    #     pedestrian_d = []
    #     list_pedestrian_sd = [pedestrian_s, pedestrian_d]
    #     for idx_pedestrian_starting in range(num_of_pedestrian):
    #         pedestrian_sd = random.choice(Pedestrian.create_starting_destination(list_position))
    #         pedestrian_s.append(pedestrian_sd[STARTING])
    #         pedestrian_d.append(pedestrian_sd[DESTINATION])
    #     return list_pedestrian_sd

    # @staticmethod
    # def create_speed(list_speed): # not used yet
    #     return list_speed
    
    # @staticmethod
    # def create_pathway(list_pedestrian_s, pedestrian_loc):
    #     for idx_pedestrian in range(num_of_pedestrian):
    #         pedestrian_s = list_pedestrian_s[idx_pedestrian]
    #         if pedestrian_s == (0,0): # generalize this
    #             pedestrian_row = pedestrian_loc[ROW]
    #             pedestrian_col = pedestrian_loc[COLUMN] + 1
    #         else:
    #             pedestrian_row = pedestrian_loc[ROW]
    #             pedestrian_col = pedestrian_loc[COLUMN] - 1
    #         pedestrian_loc = [pedestrian_row, pedestrian_col]
    #         return pedestrian_loc

    list_pedestrian_sd = Pedestrian.create_pedestrian_sd(num_of_pedestrian)
    list_pedestrian_s = list_pedestrian_sd[STARTING]
    list_pedestrian_d = list_pedestrian_sd[DESTINATION]

# draft_2_environment

class SampleEnvironment():
    
    def __init__(self, n_rows, n_cols, list_pedestrian_sd, list_pedestrian_s):
        self.time = 0
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.pedestrian_s = list_pedestrian_sd[STARTING]
        self.pedestrian_d = list_pedestrian_sd[DESTINATION]
        self.grid = self.create_location_grid(list_pedestrian_s, n_rows, n_cols)

    # @staticmethod
    # def create_pedestrian(num_of_pedestrian):
    #     pedestrian_loc = []
    #     for idx_pedestrian in range(num_of_pedestrian):
    #         pedestrian_s = list_pedestrian_s[idx_pedestrian]
    #         pedestrian_row = pedestrian_s[ROW]
    #         pedestrian_col = pedestrian_s[COLUMN]
    #         pedestrian_loc.append((pedestrian_row, pedestrian_col))
    #     return pedestrian_loc
    # consider to delete

    # @staticmethod
    # def create_location_grid(list_pedestrian_s, n_rows, n_cols):
    #     grid = []
    #     for idx_row in range(n_rows):
    #         row = []
    #         for idx_column in range(n_cols):
    #             row.append(0)
    #         grid.append(row)
    #     for idx_pedestrian in range(num_of_pedestrian):
    #         pedestrian_s = list_pedestrian_s[idx_pedestrian]
    #         grid[pedestrian_s[ROW]][pedestrian_s[COLUMN]] = 1
    #     return grid

    @staticmethod
    def update_pedestrian_location(list_pedestrian_s, pedestrian_loc):
        pedestrian_row = Pedestrian.create_pathway(list_pedestrian_s, pedestrian_loc)[ROW]
        pedestrian_col = Pedestrian.create_pathway(list_pedestrian_s, pedestrian_loc)[COLUMN]
        return pedestrian_row, pedestrian_col

    def reset(self):
        self.time = 0
        self.pedestrian_loc = self.create_pedestrian(num_of_pedestrian)
        self.grid = self.create_location_grid(self.pedestrian_s, self.n_rows, self.n_cols)
        return self.grid

    def step(self):
        self.time = self.time + 1
        self.pedestrian_loc = self.update_pedestrian_location(list_pedestrian_s, self.pedestrian_loc)
        self.grid = self.create_location_grid(list_pedestrian_s, self.n_rows, self.n_cols)
        newobs = self.grid
        for idx_pedestrian in range(num_of_pedestrian):
            if self.pedestrian_loc[idx_pedestrian] == list_pedestrian_d[idx_pedestrian]:
                done = True
            else:
                done = False
            info = {
            'goal' : list_pedestrian_d[idx_pedestrian],
            'time' : self.time
            }
        return newobs, done, info


In [71]:
env = SampleEnvironment(n_rows, n_cols, list_pedestrian_sd, list_pedestrian_s)

In [72]:
obs = env.reset()
print("obs : {}".format(obs))

obs : [[1, 0, 0, 0, 0]]


In [22]:
# draft_2 - test
# import:
import random
from itertools import product
random.seed(1)

# indexing:
STARTING = 0
DESTINATION = 1
ROW = 0
COLUMN = 1

# variables:
list_position = [(0,0), (0,4)]
num_of_pedestrian = 2
n_rows = 1
n_cols = 5

class PedestriansMaker():
    
    def __init__(self, list_position, num_of_pedestrian, n_rows, n_cols):
        self.list_position = list_position
        self.num_of_pedestrian = num_of_pedestrian
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.list_sd = self.create_starting_destination(self.list_position)
        self.list_pedestrian_sd = self.create_pedestrians_sd(self.num_of_pedestrian, self.list_sd)
        self.list_pedestrian_s = self.list_pedestrian_sd[STARTING]
        self.list_pedestrian_d = self.list_pedestrian_sd[DESTINATION]
        self.grid = self.create_location_grid(self.n_rows, self.n_cols, self.num_of_pedestrian, self.list_pedestrian_s)
        self.pedestian_pathway = self.create_pathway(self.list_pedestrian_s, self.num_of_pedestrian)
  
    @staticmethod
    def create_starting_destination(list_position):
        starting_point = list_position # since list mentioned in the init method, can we put self.list_position?
        destination_point = list_position
        list_sd = []
        for starting, destination in product(starting_point, destination_point):
            if starting != destination:
                list_sd.append((starting, destination))
        return list_sd # return a list of tuple [starting destination option]

    @staticmethod
    def create_pedestrians_sd(num_of_pedestrian, list_sd):
        pedestrian_s = []
        pedestrian_d = []
        for idx_pedestrian_starting in range(num_of_pedestrian):
            pedestrian_sd = random.choice(list_sd)
            pedestrian_s.append(pedestrian_sd[STARTING])
            pedestrian_d.append(pedestrian_sd[DESTINATION])
            list_pedestrian_sd = [pedestrian_s, pedestrian_d]
        return list_pedestrian_sd # return list of tuple [starting][destination]

    @staticmethod
    def create_location_grid(n_rows, n_cols, num_of_pedestrian, list_pedestrian_s):
        grid = []
        for idx_row in range(n_rows):
            row = []
            for idx_column in range(n_cols):
                row.append(0)
            grid.append(row)
        for idx_pedestrian in range(num_of_pedestrian):
            pedestrian_s = list_pedestrian_s[idx_pedestrian]
            grid[pedestrian_s[ROW]][pedestrian_s[COLUMN]] = 1
        return grid # return the grid and each pedestrian position on the grid
    
    @staticmethod
    def create_pathway(list_pedestrian_s, num_of_pedestrian):
        pedestrian_pathway = []
        for idx_pedestrian in range(num_of_pedestrian):
            pedestrian_s = list_pedestrian_s[idx_pedestrian]
            if pedestrian_s == (0,0): # will later generalize this
                row_update = 0
                col_update = 1
            else:
                row_update = 0
                col_update = -1
            pedestrian_pathway.append((row_update, col_update))
        return pedestrian_pathway

class StreetLight():
    pass

class SampleEnvironment():

    def __init__(self, list_position, num_of_pedestrian, n_rows, n_cols):
        self.list_position = list_position
        self.num_of_pedestrian = num_of_pedestrian
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.time = 0
        self.PedestriansMaker_class = PedestriansMaker(self.list_position, self.num_of_pedestrian, self.n_rows, self.n_cols)
        self.list_pedestrian_s = self.PedestriansMaker_class.list_pedestrian_s
        self.list_pedestrian_d = self.PedestriansMaker_class.list_pedestrian_d
        self.grid = self.PedestriansMaker_class.grid

    def reset(self):
        self.time = 0
        self.list_pedestrian_s = self.PedestriansMaker_class.list_pedestrian_s
        self.grid = self.PedestriansMaker_class.grid
        return self.grid

    def step(self):
        self.time = self.time + 1
        self.list_pedestrian_loc = self.PedestriansMaker_class.pedestian_loc
        self.grid = self.PedestriansMaker_class.grid
        newobs = self.grid
        for idx_pedestrian in range(self.num_of_pedestrian):
            pedestrian_loc = self.list_pedestrian_loc[idx_pedestrian]
            pedestrian_destination = self.list_pedestrian_d[idx_pedestrian]
            if pedestrian_loc == pedestrian_destination:
                done = True
            else:
                done = False
            info = {
                'goal' : pedestrian_destination,
                'time' : self.time
            }
        return newobs, done, info

    def render(self):
        for rows in self.grid:
            print(rows)        

a = PedestriansMaker(list_position, num_of_pedestrian, n_rows, n_cols)
print('starting destination combination = {}'.format(a.create_starting_destination(list_position)))
print('pedestrian starting destination combination = {}'.format(a.create_pedestrians_sd(num_of_pedestrian, a.list_sd)))
print('pedestrian location starting = {}'.format(a.create_location_grid(n_rows, n_cols, num_of_pedestrian, a.list_pedestrian_s)))
print('pedestrian starting destination combination = {}'.format(a.list_pedestrian_sd))
print('pedestrian starting = {}'.format(a.list_pedestrian_s))
print('pedestrian destination = {}'.format(a.list_pedestrian_d))
print('pedestrian pathway = {}'.format(a.pedestian_pathway))
print('done')
b = SampleEnvironment(list_position, num_of_pedestrian, n_rows, n_cols)
print('test')
obs = b.reset()
print(obs)

starting destination combination = [((0, 0), (0, 4)), ((0, 4), (0, 0))]
pedestrian starting destination combination = [[(0, 4), (0, 0)], [(0, 0), (0, 4)]]
pedestrian location starting = [[1, 0, 0, 0, 0]]
pedestrian starting destination combination = [[(0, 0), (0, 0)], [(0, 4), (0, 4)]]
pedestrian starting = [(0, 0), (0, 0)]
pedestrian destination = [(0, 4), (0, 4)]
pedestrian pathway = [(0, 1), (0, 1)]
done
test
[[0, 0, 0, 0, 1]]


In [ ]:

class StreetLight():

    def __init__():
        pass

    def create_sl():
        pass
